In [3]:
!pip install torch
import torch

  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
   ---------------------------------------- 0.0/198.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/198.5 MB 10.6 MB/s eta 0:00:19
   ---------------------------------------- 1.0/198.5 MB 13.3 MB/s eta 0:00:15
   ---------------------------------------- 1.7/198.5 MB 13.4 MB/s eta 0:00:15
   ---------------------------------------- 2.4/198.5 MB 15.2 MB/s eta 0:00:13
    --------------------------------------- 3.1/198.5 MB 15.1 MB/s eta 0:00:13
    --------------------------------------- 3.8/198.5 MB 16.1 MB/s eta 0:00:13
    --------------------------------------- 4.9/198.5 MB 17.3 MB/s eta 0:00:12
   - -------------------------------------- 5.7/198.5 MB 17.4 MB/s eta 0:00:12
   - -------------------------------------- 6.9/198.5 MB 18.4 MB/s eta 0:00:11
   - -------------------------------------- 8.2/198.5 MB 19.3 MB/s eta 0:00:10
   - -------------------------------------- 9.6/198.5 MB 20.3 MB/s eta 0:00:

c:\USERS\HP\DESKTOP\DLCV\DLCV\MNIST\ENV\Lib\site-packages\torch\nn\modules\transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [3]:
!nvidia-smi

Sun Apr 14 02:06:20 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.78                 Driver Version: 551.78         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce MX110         WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A    0C    P8             N/A /  200W |       0MiB /   2048MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
tensor=torch.rand(1,3,4)
tensor

tensor([[[0.1900, 0.2115, 0.1153, 0.7070],
         [0.0300, 0.3769, 0.4199, 0.1668],
         [0.8227, 0.1048, 0.9594, 0.3902]]])

In [6]:
torch.tensor([3,4],device='cuda')

tensor([3, 4], device='cuda:0')

In [7]:
t = torch.rand(5, 3)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
t = t.to(device)

In [8]:
print(tensor.get_device())

-1


In [9]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce MX110'

In [10]:
torch.cuda.device_count()


1

In [11]:
torch.cuda.current_device()


0

In [12]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce MX110
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [13]:
import torch
import torch.nn as nn
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
t1 = torch.randn(1,2)
t2 = torch.randn(1,2).to(dev)
print(t1)  # tensor([[-0.2678,  1.9252]])
print(t2)  # tensor([[ 0.5117, -3.6247]], device='cuda:0')
t1.to(dev)
print(t1)  # tensor([[-0.2678,  1.9252]])
print(t1.is_cuda) # False
t1 = t1.to(dev)
print(t1)  # tensor([[-0.2678,  1.9252]], device='cuda:0')
print(t1.is_cuda) # True

class M(nn.Module):
    def __init__(self):        
        super().__init__()        
        self.l1 = nn.Linear(1,2)

    def forward(self, x):                      
        x = self.l1(x)
        return x
model = M()   # not on cuda
model.to(dev) # is on cuda (all parameters)
print(next(model.parameters()).is_cuda) # True

tensor([[-1.7511, -0.8084]])
tensor([[1.2000, 0.1877]], device='cuda:0')
tensor([[-1.7511, -0.8084]])
False
tensor([[-1.7511, -0.8084]], device='cuda:0')
True
True


In [14]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transform data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Define CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Train the network
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# Test the network
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


100%|██████████| 170498071/170498071 [00:17<00:00, 9664173.05it/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
[1,  2000] loss: 2.211
[1,  4000] loss: 1.870
[1,  6000] loss: 1.721
[1,  8000] loss: 1.595
[1, 10000] loss: 1.541
[1, 12000] loss: 1.510
[2,  2000] loss: 1.432
[2,  4000] loss: 1.396
[2,  6000] loss: 1.367
[2,  8000] loss: 1.337
[2, 10000] loss: 1.307
[2, 12000] loss: 1.293
Finished Training
Accuracy of the network on the 10000 test images: 55 %
